In [3]:
import json
import operator
import os

In [2]:
Xing = json.load(open('Pure_Xing_Final.json'))

In [9]:
path = '/home/xing/xing/Recsys2018_NEW/WAR/0630_FINAL/Pure_QQ/'
QQ_ALL = []
for filename in os.listdir(path):
    QQ_ALL.append(json.load(open(path + filename)))

In [10]:
W2V = json.load(open('Pure_W2V_Final.json'))

In [11]:
PLSIM_W2V = json.load(open('Pure_PLSIMW2V_Final.json'))

## Challenge Seed

In [12]:
WAR_SEED = json.load(open('/home/xing/xing/Recsys2018_NEW/DATA_PROCESSING/WAR_PL_TRACKS_READONLY.json'))

In [13]:
WAR_HOLDOUT = json.load(open('/home/xing/xing/Recsys2018_NEW/DATA_PROCESSING/WAR_PL_TRACKS_READONLY_HOLDOUT.json'))

## Combine Functions

In [14]:
def rank(new, current, sort_base):
    layer = list(set(new) - set(current))
    layer_index = {}
    for elem in layer:
        layer_index[elem] = sort_base[elem]
        
    layer_sorted = sorted(layer_index.items(), key=operator.itemgetter(1))
    layer_final = [i[0] for i in layer_sorted]
    return current + layer_final

In [15]:
def combine_method_T1_old(QQ, Xing, GT_len):
    QQ_index = {}
    for elem in QQ:
        QQ_index[elem] = len(QQ_index)
    
    Xing_index = {}
    for elem in Xing:
        Xing_index[elem] = len(Xing_index)
        
        
    Ax = set(Xing[:GT_len])
    Bx = set(Xing[GT_len:])
        
    Aq = set(QQ[:GT_len])
    Bq = set(QQ[GT_len:])
    
    # Layer 1
    up_to_layer1 = rank(list(Ax & Aq), [], Xing_index)
    
    # Layer 2 part 1
    up_to_layer2_1 = rank(list(Ax & Bq), up_to_layer1, Xing_index)

    # Layer 2 part 2
    up_to_layer2_2 = rank(list(Ax), up_to_layer2_1, Xing_index)
    
    # Layer 3 part 1
    up_to_layer3_1 = rank(list(Bx & Aq), up_to_layer2_2, Xing_index)
    
    # Layer 3 part 2
    up_to_layer3_2 = rank(list(Aq), up_to_layer3_1, QQ_index)
    
    # Layer 3 part 3
    up_to_layer3_3 = rank(list(Bx & Bq), up_to_layer3_2, Xing_index)
    
    # Layer 4 part 1
    up_to_layer4_1 = rank(list(Bx), up_to_layer3_3, Xing_index)
    
    # Layer 4 part 2
    up_to_layer4_2 = rank(list(Bq), up_to_layer4_1, QQ_index)
    
    return up_to_layer4_2[:500]

In [27]:
def combine_method_T1_new(QQ_ALL, Xing, GT_len):
    all_tracks_qq = []
    for each in QQ_ALL:
        all_tracks_qq += each
    all_tracks_qq = list(set(all_tracks_qq))
    
    QQ_ALL_index = []
    for each in QQ_ALL:
        current = {}
        t = 0
        for elem in each:
            current[elem] = (501 - t)
            t += 1
        QQ_ALL_index.append(current)
    
    for elem in all_tracks_qq:
        for each_qq_index in QQ_ALL_index:   
            if elem not in each_qq_index:
                each_qq_index[elem] = 0
        
    final_index = {}
    for elem in all_tracks_qq:
        final_index[elem] = 0
        for each_qq_index in QQ_ALL_index: 
            final_index[elem] += each_qq_index[elem]
        
    layer_sorted = sorted(final_index.items(), key=operator.itemgetter(1), reverse = True)
    layer_final = [i[0] for i in layer_sorted] 
    
    final_qq = layer_final[:500]
    
    return combine_method_T1_old(final_qq, Xing, GT_len)

In [28]:
def combine_method_T2_T9(QQ_ALL, Xing, W2V, PLSIM_W2V, GT_len, parameters):  
    
    all_tracks_qq = []
    for each in QQ_ALL:
        all_tracks_qq += each
    all_tracks_qq = list(set(all_tracks_qq))
    
    all_tracks = list(set(all_tracks_qq + Xing + W2V + PLSIM_W2V))
    
    ALL_index = []
    i = 0
    for each in QQ_ALL:
        current = {}
        t = 0
        for elem in each:
            current[elem] = (501 - t) * float(parameters[i])
            t += 1
        i += 1
        ALL_index.append(current)
        
    Xing_index = {}
    t = 0
    for elem in Xing:
        Xing_index[elem] = (501 - t) * float(parameters[-3])
        t += 1   
    ALL_index.append(Xing_index)
        
    W2V_index = {}
    t = 0
    for elem in W2V:
        W2V_index[elem] = (501 - t) * float(parameters[-2])
        t += 1
    ALL_index.append(W2V_index)
        
    PLSIM_W2V_index = {}
    t = 0
    for elem in PLSIM_W2V:
        PLSIM_W2V_index[elem] = (501 - t) * float(parameters[-1])
        t += 1
    ALL_index.append(PLSIM_W2V_index)
            
    for elem in all_tracks:
        for each_index in ALL_index:
            if elem not in each_index:
                each_index[elem] = 0
        
    final_index = {}
    for elem in all_tracks:
        
        final_index[elem] = 0
        for each_index in ALL_index:
            final_index[elem] += each_index[elem]
            
    layer_sorted = sorted(final_index.items(), key=operator.itemgetter(1), reverse = True)
    layer_final = [i[0] for i in layer_sorted] 
    
    return layer_final[:500]

## Combine Now

In [29]:
Parameters = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, \
              0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, \
              0.1, 0.1, 0.1, 0.1, 0.1, 2.75, 0.0, 0.0] # new T1 

## Trailmix Main

In [30]:
R_combined = {}
for pl in QQ_ALL[0]:
    current_qq_all = []
    for each_qq in QQ_ALL:
        current_qq_all.append(each_qq[pl])
    if len(WAR_SEED[pl]) == 1:
        R_combined[pl] = combine_method_T1_new(current_qq_all, Xing[pl],WAR_HOLDOUT[pl])
    else:
        R_combined[pl] = combine_method_T2_T9(current_qq_all, Xing[pl], W2V[pl], PLSIM_W2V[pl], WAR_HOLDOUT[pl], Parameters)

T0 = json.load(open('T0/WAR_T0_MODEL1_MOST_POPULAR_BY_WORD.json')) # old T0

R_final = dict(R_combined.items() + T0.items())

In [31]:
with open('Submission/Submission_0630_Trailmix_main_fake.csv', 'w') as f:
    f.write('team_info,')
    f.write('main,')
    f.write('Trailmix,')
    f.write('zhaoxing623@email.tamu.edu\n')
    
    for pid in R_final:
        f.write(pid)
        for track in R_final[pid]:
            f.write(',' + 'spotify:track:' + track)
        f.write('\n')

## Trailmix Creative

In [32]:
R_combined = {}
for pl in QQ_ALL[0]:
    current_qq_all = []
    for each_qq in QQ_ALL:
        current_qq_all.append(each_qq[pl])
    if len(WAR_SEED[pl]) == 1:
        R_combined[pl] = combine_method_T1_new(current_qq_all, Xing[pl],WAR_HOLDOUT[pl])
    else:
        R_combined[pl] = combine_method_T2_T9(current_qq_all, Xing[pl], W2V[pl], PLSIM_W2V[pl], WAR_HOLDOUT[pl], Parameters)

T0 = json.load(open('T0/WAR_T0_MODEL1_MOST_POPULAR_BY_WORD_0628XY.json')) # new T0

R_final = dict(R_combined.items() + T0.items())

In [33]:
with open('Submission/Submission_0630_Trailmix_creative_fake.csv', 'w') as f:
    f.write('team_info,')
    f.write('creative,')
    f.write('Trailmix,')
    f.write('zhaoxing623@email.tamu.edu\n')
    
    for pid in R_final:
        f.write(pid)
        for track in R_final[pid]:
            f.write(',' + 'spotify:track:' + track)
        f.write('\n')